In [1]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import time

# import functions for ODE parameter estimation
from model.diag_Gaussian import *

np.random.seed(123)

## Define ODE (x, time, parameters)

In [2]:
# define transform function to constrain parameter values
def transform(params):
    
    # unpack parameters
    mu_max, pref, Sigma_inv, K_inv, delta, gamma = params
    
    # take exp of strictly positive parameters
    mu_max = jnp.exp2(mu_max) 
    pref = jnp.exp2(pref)*7.
    Sigma_inv = jnp.exp2(Sigma_inv)/10.
    K_inv = jnp.exp2(K_inv)/10.
    delta = jnp.exp2(delta)/10.
    gamma = gamma/10.
    
    return jnp.array([mu_max, pref, Sigma_inv, K_inv, delta, gamma])

# ode 
def system(x, t, params): 

    # unpack parameters
    mu_max, pref, Sigma_inv, K_inv, delta, gamma = transform(params)
    
    # unpack variables
    od, pH = x 
           
    # coupled ODE for cells and environmental pH
    dod = od * (mu_max - K_inv * od) * (jnp.exp2(-Sigma_inv * (pH - pref)**2.) - delta) 
    # dpH = (gamma * od) * (pH * (2. - pH / 7.))
    dpH = (gamma * dod) * (pH * (1. - pH / 14.))
    
    return jnp.array([dod, dpH])  

# define prior mean of untransformed variables 
prior_mean = np.array([0., 0., 0., 0., 0., 0.])

## Import data as a Pandas dataframe 

The dataframe must have a "Treatments" column that provides a unique identifier for each condition. The second column specifies the time point that observations were made. The following columns include the state variables.

In [3]:
# import data
df = pd.read_csv("Data/arc_allmono.csv").rename(columns={"Experiments":"Treatments"})

In [4]:
# species names
all_species = df.columns.values[3:]
all_species

array(['AC_OD', 'BA_OD', 'BC_OD', 'BL_OD', 'BT_OD', 'BV_OD', 'CC_OD',
       'DF_OD', 'wt_OD', 'delarc_OD', 'parc-_OD', 'parc+_OD'],
      dtype=object)

In [6]:
# loop over monocultures
for sp in all_species:
    
    # init list of dataframes of model predictions
    df_preds = []

    # pH vals
    ph_vals = [4, 5, 6, 7, 8, 9]
    
    # loop over ph values
    for ph_val in ph_vals:
    
        # names of species in ODE
        species = [sp, 'pH']
    
        # pull out train and test dataframes
        df_train = pd.concat([df_exp for exp_name, df_exp in df.groupby("Treatments") if sp.split("_")[0] in exp_name and str(ph_val) not in exp_name])
        df_test  = pd.concat([df_exp for exp_name, df_exp in df.groupby("Treatments") if sp.split("_")[0] in exp_name and str(ph_val) in exp_name])
        
        # average test data over time
        df_groups = df_test.groupby("Time")
        df_test = df_groups[species].mean().reset_index()
        df_test.insert(0, "Treatments", [sp.split("_")[0]+"_"+str(ph_val)]*df_test.shape[0])
        
        # instantiate gLV fit 
        ode_model = ODE(system = system, 
                    transform = transform,
                    dataframe = df_train, 
                    sys_vars = species, 
                    prior_mean = prior_mean, 
                    alpha=10.)

        # fit to data 
        ode_model.fit_posterior_EM(n_sample_sgd=1, 
                                   n_sample_hypers=50, 
                                   n_sample_evidence=100, 
                                   patience=3)


        # predict values and save to test dataframe
        x0 = df_test[species].values[0]
        y_preds = ode_model.predict_sample(x0, np.array(df_test.Time.values, float), n_sample=100)
        
        # save predictions 
        df_test.drop(columns='pH', inplace=True)   
        df_test[[f'sample_{i+1}' for i in range(y_preds.shape[0])]] = y_preds[:, :, 0].T
        
        # append to list of dataframes
        df_preds.append(df_test)
        
    # save dataframe
    df_pred = pd.concat(df_preds)
    df_pred.to_csv("kfold/{:}.csv".format(sp.split("_")[0]), index=False)

Trial 1
Loss: 9094.475
Loss: 5544.635
Loss: 3597.885
Loss: 1401.569
Loss: 1327.256
Loss: 1204.328
Loss: 1115.406
Loss: 1066.718
Loss: 989.862
Loss: 944.140
Loss: 911.903
Loss: 772.699
Loss: 756.354
Loss: 749.032
Loss: 736.065
Loss: 648.497
Loss: 639.178
Loss: 637.904
Loss: 635.713
Loss: 631.803
Loss: 626.741
Loss: 626.697
Loss: 623.145
Loss: 622.553
Loss: 622.453
Loss: 622.329
Loss: 619.842
Loss: 619.674
Loss: 619.645
Loss: 617.255
Loss: 616.832
Loss: 616.747
Loss: 616.648
Loss: 614.656
Loss: 614.329
Loss: 614.264
Loss: 614.204
Loss: 612.609
Loss: 611.995
Loss: 611.868
Loss: 611.804
Loss: 610.558
Loss: 610.366
Loss: 610.326
Loss: 610.301
Loss: 609.391
Loss: 609.314
Loss: 609.298
Loss: 608.712
Loss: 608.686
Loss: 608.680
Loss: 608.133
Loss: 607.803
Loss: 607.730
Loss: 607.679
Loss: 607.677
Loss: 607.383
Loss: 607.374
Loss: 607.372
Loss: 607.114
Loss: 607.104
Loss: 607.103
Loss: 606.974
Loss: 606.974
Loss: 606.875
Loss: 606.874
Loss: 606.872
Loss: 606.800
Loss: 606.799
Trial 2
Trial 3
Lo

100%|███████████████████████████████████████████| 50/50 [00:00<00:00, 56.57it/s]


Updating posterior...
Epoch 1, NEG ELBO: 102.061
Epoch 2, NEG ELBO: 111.802
Epoch 3, NEG ELBO: 104.821
Epoch 4, NEG ELBO: 106.596
Epoch 5, NEG ELBO: 103.242
Epoch 6, NEG ELBO: 103.567
Epoch 7, NEG ELBO: 105.878
Epoch 8, NEG ELBO: 104.092
Epoch 9, NEG ELBO: 102.362
Epoch 10, NEG ELBO: 101.684
Epoch 11, NEG ELBO: 105.647, Slope: -0.005
Epoch 12, NEG ELBO: 103.497, Slope: -0.002
Epoch 13, NEG ELBO: 103.140, Slope: -0.002
decrease lr to 9.000e-04
Epoch 14, NEG ELBO: 108.697, Slope: 0.002
decrease lr to 8.100e-04
Epoch 15, NEG ELBO: 123.174, Slope: 0.010
decrease lr to 7.290e-04
Epoch 16, NEG ELBO: 102.090, Slope: 0.007
decrease lr to 6.561e-04
Epoch 17, NEG ELBO: 103.498, Slope: 0.005
decrease lr to 5.905e-04
Epoch 18, NEG ELBO: 102.960, Slope: 0.003
Epoch 19, NEG ELBO: 101.996, Slope: -0.000
Epoch 20, NEG ELBO: 101.905, Slope: -0.004
Epoch 21, NEG ELBO: 103.957, Slope: -0.005
Epoch 22, NEG ELBO: 105.570, Slope: -0.006
Epoch 23, NEG ELBO: 102.700, Slope: -0.009
Epoch 24, NEG ELBO: 101.678,

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 59.06it/s]


Log evidence: -1012.516 +/- 0.000
Updating hyperparameters...


...

100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 328.57it/s]


Updating posterior...
Epoch 1, NEG ELBO: 85.719
Epoch 2, NEG ELBO: 87.916
Epoch 3, NEG ELBO: 94.524
Epoch 4, NEG ELBO: 85.531
Epoch 5, NEG ELBO: 85.411
Epoch 6, NEG ELBO: 85.473
Epoch 7, NEG ELBO: 84.687
Epoch 8, NEG ELBO: 92.177
Epoch 9, NEG ELBO: 85.026
Epoch 10, NEG ELBO: 85.234
Epoch 11, NEG ELBO: 84.676, Slope: -0.005
Epoch 12, NEG ELBO: 88.183, Slope: -0.004
decrease lr to 9.000e-04
Epoch 13, NEG ELBO: 85.723, Slope: 0.001
Epoch 14, NEG ELBO: 84.405, Slope: -0.001
decrease lr to 8.100e-04
Epoch 15, NEG ELBO: 88.665, Slope: 0.000
decrease lr to 7.290e-04
Epoch 16, NEG ELBO: 97.451, Slope: 0.006
decrease lr to 6.561e-04
Epoch 17, NEG ELBO: 87.076, Slope: 0.003
decrease lr to 5.905e-04
Epoch 18, NEG ELBO: 85.043, Slope: 0.005
decrease lr to 5.314e-04
Epoch 19, NEG ELBO: 85.463, Slope: 0.002
decrease lr to 4.783e-04
Epoch 20, NEG ELBO: 96.236, Slope: 0.006
decrease lr to 4.305e-04
Epoch 21, NEG ELBO: 87.438, Slope: 0.003
decrease lr to 3.874e-04
Epoch 22, NEG ELBO: 87.076, Slope: 0.0

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 87.73it/s]

Log evidence: -327.564 +/- 0.000



/var/folders/c2/p6l4gf2935z26c2jq5cy44b00000gn/T/ipykernel_16770/2648872324.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[[f'sample_{i+1}' for i in range(y_preds.shape[0])]] = y_preds[:, :, 0].T
/var/folders/c2/p6l4gf2935z26c2jq5cy44b00000gn/T/ipykernel_16770/2648872324.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[[f'sample_{i+1}' for i in range(y_preds.shape[0])]] = y_preds[:, :, 0].T
/var/folders/c2/p6l4gf2935z26c2jq5cy44b00000gn/T/ipykernel_16770/2648872324.py:46: PerformanceWarning: DataFrame 